#  AI Graphics | NYU 2024
### Name: Qingxin Meng

## Assignment - Deform a source mesh to form a target mesh using 3D loss functions

This assignment was tested in my local machine instead of Google Colab. All the discussions are based on the outputs from my notebook execution. Since the pytorch3d optimization involves randomness, there might be a huge difference compared to my notebook if you run it again. Additionally, there are strikethrough lines in some discussions which explain phenomena in outputs from different executions.

If you decide to get similar outputs to mine, you should always try "Restart kernel and run all cells." This whole notebook takes 800 seconds to run on a 4070ti GPU. 

## 0. Install and Import modules

Ensure `torch` and `torchvision` are installed. If `pytorch3d` is not installed, install it using the following cell:

In [ ]:
import os
import sys
import torch
need_pytorch3d=False
try:
    import pytorch3d
except ModuleNotFoundError:
    need_pytorch3d=True
if need_pytorch3d:
    if torch.__version__.startswith(("2.2.")) and sys.platform.startswith("linux"):
        # We try to install PyTorch3D via a released wheel.
        pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
        version_str="".join([
            f"py3{sys.version_info.minor}_cu",
            torch.version.cuda.replace(".",""),
            f"_pyt{pyt_version_str}"
        ])
        !pip install fvcore iopath
        !pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
    else:
        # We try to install PyTorch3D from source.
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

In [ ]:
import os
import torch
from pytorch3d.io import load_obj, save_obj,load_objs_as_meshes
from pytorch3d.structures import Meshes
from pytorch3d.utils import ico_sphere,torus
from pytorch3d.ops import sample_points_from_meshes
from pytorch3d.loss import (
    chamfer_distance,
    mesh_edge_loss,
    mesh_laplacian_smoothing,
    mesh_normal_consistency,
)
from pytorch3d.transforms import RotateAxisAngle
import numpy as np
from tqdm.notebook import tqdm
%matplotlib notebook
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['savefig.dpi'] = 80
mpl.rcParams['figure.dpi'] = 80

from plotly.subplots import make_subplots
torch.manual_seed(777)

# Set the device
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
    print("WARNING: CPU only, this will be slow!")

## 1. Load an obj file and create a Meshes object

Download the target 3D model of a dolphin. It will be saved locally as a file called `dolphin.obj`.

In [ ]:
!wget -nc https://dl.fbaipublicfiles.com/pytorch3d/data/dolphin/dolphin.obj

In [ ]:
# Load the dolphin mesh.
trg_obj = os.path.join('dolphin.obj')

In [ ]:
# We read the target 3D model using load_obj
verts, faces, aux = load_obj(trg_obj)

# verts is a FloatTensor of shape (V, 3) where V is the number of vertices in the mesh
# faces is an object which contains the following LongTensors: verts_idx, normals_idx and textures_idx
# For this tutorial, normals and textures are ignored.
faces_idx = faces.verts_idx.to(device)
verts = verts.to(device)

In [ ]:
# We initialize the source shape to be a sphere of radius 1
src_mesh = ico_sphere(4, device)

1.1 The source mesh is a sphere of radius 1 centered at (0, 0, 0). To speed up the optimization process, we should scale normalize the target mesh and center it at the origin, making it fit the unit sphere. Do this operations with the `verts`tensor and create a `Meshes` object called "trg_mesh" with the normalized vertices and the faces indices.

1.2 Visualize the source and target meshes using `Plotly` integration.

In [ ]:
##############################################################################
# Code for 1.1
centered_verts = verts-verts.mean()
largest_distance=centered_verts.norm(dim=1).max()
normalized_verts = centered_verts/largest_distance

trg_mesh=Meshes([normalized_verts], [faces_idx])


# Code for 1.2
import plotly.graph_objects as go
import numpy as np


def plotly_submeshes(verts,faces,size):
    specs=[[{"type": "mesh3d"} for i in range(size[1])] for j in range(size[0])]
    fig = make_subplots(size[0],size[1],specs=specs)
    for j in range(size[0]):
        for i in range(size[1]):
            fig.add_trace(go.Mesh3d(x=verts[j][i][:,0],y=verts[j][i][:,1],z=verts[j][i][:,2],i=faces[j][i][:,0],j=faces[j][i][:,1],k=faces[j][i][:,2]),row=j+1,col=i+1)
    return fig


verts1=trg_mesh.verts_packed().cpu()
faces1=trg_mesh.faces_packed().cpu()
#fig1 = go.Figure(data=[go.Mesh3d(x=verts1[:,0], y=verts1[:,1], z=verts1[:,2],i=faces1[:, 0],j=faces1[:, 1],k=faces1[:, 2])])
#fig1.show()
verts5=verts.cpu()

verts2=src_mesh.verts_packed().cpu()
faces2=src_mesh.faces_packed().cpu()
#fig2 =go.Figure(data=[go.Mesh3d(x=verts2[:,0], y=verts2[:,1], z=verts2[:,2],i=faces2[:, 0],j=faces2[:, 1],k=faces2[:, 2])])
#fig2.show()


fig1 = plotly_submeshes([[verts1,verts2]],[[faces1,faces2]],[1,2])
fig1.show()
fig3 = go.Figure(data=[go.Mesh3d(x=verts1[:,0], y=verts1[:,1], z=verts1[:,2],i=faces1[:, 0],j=faces1[:, 1],k=faces1[:, 2]),go.Mesh3d(x=verts2[:,0], y=verts2[:,1], z=verts2[:,2],i=faces2[:, 0],j=faces2[:, 1],k=faces2[:, 2],opacity=0.5)])
fig3.show()
##############################################################################

###  Visualizing point clouds with Matplotlib

We can also sample points from the surfaces and visualize them using Matplotlib. It can be a useful to have a coarse estimation of the surface during iterations in the optimization process.

In [ ]:
def plot_pointcloud(mesh, title=""):
    # Sample points uniformly from the surface of the mesh.
    points = sample_points_from_meshes(mesh, 5000)
    x, y, z = points.clone().detach().cpu().squeeze().unbind(1)
    fig = plt.figure(figsize=(5, 5))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter3D(x, z, -y)
    ax.set_xlabel('x')
    ax.set_ylabel('z')
    ax.set_zlabel('y')
    ax.set_title(title)
    ax.view_init(190, 30)
    plt.show()

1.3 Use the function `plot_pointcloud` to see the initial pointcloud

In [ ]:
%matplotlib inline
##############################################################################
# Code for 1.3
plot_pointcloud(src_mesh,"src_mesh")
plot_pointcloud(trg_mesh,"trg_mesh")
##############################################################################

## 2. Optimization loop

Starting from a sphere mesh, we learn the offset to each vertex in the mesh such that the predicted mesh is closer to the target mesh at each optimization step. To achieve this we minimize the distance between the predicted (deformed) and target mesh, defined as the **chamfer distance** between the set of pointclouds resulting from **differentiably sampling points** from their surfaces.

However, solely minimizing the chamfer distance between the predicted and the target mesh will lead to a non-smooth shape. We enforce smoothness by adding **shape regularizers** to the objective. Namely, we add:

+ `mesh_edge_length`, which minimizes the length of the edges in the predicted mesh.
+ `mesh_normal_consistency`, which enforces consistency across the normals of neighboring faces.
+ `mesh_laplacian_smoothing`, which is the laplacian regularizer.
We will learn to deform the source mesh by offsetting its vertices

In [ ]:
# The shape of the deform parameters is equal to the total number of vertices in src_mesh
deform_verts = torch.full(src_mesh.verts_packed().shape, 0.0, device=device, requires_grad=True)

In [ ]:
# The optimizer
optimizer = torch.optim.SGD([deform_verts], lr=1.0, momentum=0.9)

OBS: Note that you need the "trg_mesh" from 1.1 for the optimization code to work.

In [ ]:
# Number of optimization steps
Niter = 2000
# Weight for the chamfer loss
w_chamfer = 1.0
# Weight for mesh edge loss
w_edge = 1.0
# Weight for mesh normal consistency
w_normal = 0.01
# Weight for mesh laplacian smoothing
w_laplacian = 0.1
# Plot period for the losses
plot_period = 250
loop = tqdm(range(Niter))

chamfer_losses = []
laplacian_losses = []
edge_losses = []
normal_losses = []
total_losses=[]
%matplotlib inline

for i in loop:
    # Initialize optimizer
    optimizer.zero_grad()

    # Deform the mesh
    new_src_mesh = src_mesh.offset_verts(deform_verts)

    # We sample 5k points from the surface of each mesh
    sample_trg = sample_points_from_meshes(trg_mesh, 5000)
    sample_src = sample_points_from_meshes(new_src_mesh, 5000)

    # We compare the two sets of pointclouds by computing (a) the chamfer loss
    loss_chamfer, _ = chamfer_distance(sample_trg, sample_src)

    # and (b) the edge length of the predicted mesh
    loss_edge = mesh_edge_loss(new_src_mesh)

    # mesh normal consistency
    loss_normal = mesh_normal_consistency(new_src_mesh)

    # mesh laplacian smoothing
    loss_laplacian = mesh_laplacian_smoothing(new_src_mesh, method="uniform")

    # Weighted sum of the losses
    loss = loss_chamfer * w_chamfer + loss_edge * w_edge + loss_normal * w_normal + loss_laplacian * w_laplacian

    # Print the losses
    loop.set_description('total_loss = %.6f' % loss)

    # Save the losses for plotting
    chamfer_losses.append(float(loss_chamfer.detach().cpu()))
    edge_losses.append(float(loss_edge.detach().cpu()))
    normal_losses.append(float(loss_normal.detach().cpu()))
    laplacian_losses.append(float(loss_laplacian.detach().cpu()))
    total_losses.append(float(loss.detach().cpu()))

    # Plot mesh
    if i % plot_period == 0:
        plot_pointcloud(new_src_mesh, title="iter: %d" % i)

    # Optimization step
    loss.backward()
    optimizer.step()


## Visualize the loss

In [ ]:
def plot_losses(loss_dict, size=(13,5)):
  fig = plt.figure(figsize=size)
  ax = fig.gca()
  for loss_name, loss_values in loss_dict.items():
    ax.plot(loss_values, label=loss_name)
  ax.legend(fontsize="16")
  ax.set_xlabel("Iteration", fontsize="16")
  ax.set_ylabel("Loss", fontsize="16")
  ax.set_title("Loss vs iterations", fontsize="16");

In [ ]:
losses = {"chamfer loss": chamfer_losses,
          "edge loss": edge_losses,
          "normal loss": normal_losses,
          "laplacian loss": laplacian_losses,
          "total loss":total_losses}
plot_losses(losses, (18, 10))

## Save the predicted mesh

In [ ]:
# Fetch the verts and faces of the final predicted mesh
final_verts, final_faces = new_src_mesh.get_mesh_verts_faces(0)

2.1 Scale normalize back the `final_verts` to the original target size



In [ ]:
##############################################################################
# Code for 2.1
# a modification in previous block to remember the size
final_verts_nback=final_verts*largest_distance
##############################################################################

In [ ]:
# Store the predicted mesh using save_obj
final_obj = os.path.join('./', 'final_model.obj')
#save_obj(final_obj, final_verts_nback, final_faces)

2.2 Visualize the target mesh and the final mesh using `Plotly`. Qualitatively, what do you think of the result?

2.3 Take a look at the **Loss vs Iteration** graph, paying attention to how the losses are decreasing. Do you think it would be possible to achieve a reasonable result with less iterations? Is it worth to run with more iterations? Explain you thoughts, then run the optimization loop again making the changes to validate your hypothesis.

2.4 The loss function used for this task is a linear combination of four losses. Let `w_chamfer = 1.0` and set the other coefficients to zero. Run the  optimization loop again with this configuration and describe the result.

**[EXTRA] E.1 Experiment other coefficients configurations and describe the results.**

2.5 Experiment with others optimizers such as `Adam` and `RMSprop` in place of `SGD`. Also, try changing the learning rate (`lr`), and `momentum` when these parameters are applied. What do you observe in terms of speed of convergence and quality of final results?




In [ ]:
##############################################################################
# Code for 2.2

verts4=final_verts_nback.detach().cpu()
faces4=final_faces.cpu()
#fig4 =go.Figure(data=[go.Mesh3d(x=verts4[:,0], y=verts4[:,1], z=verts4[:,2],i=faces4[:, 0],j=faces4[:, 1],k=faces4[:, 2])])
#fig4=make_subplots(1,2,specs=[[{"type": "mesh3d"}, {"type": "mesh3d"}]])
#fig4.add_trace(go.Mesh3d(x=verts4[:,0], y=verts4[:,1], z=verts4[:,2],i=faces4[:, 0],j=faces4[:, 1],k=faces4[:, 2]), row=1, col=1)

#fig4.add_trace(go.Mesh3d(x=verts5[:,0], y=verts5[:,1], z=verts5[:,2],i=faces1[:, 0],j=faces1[:, 1],k=faces1[:, 2]), row=1, col=2)
fig4=plotly_submeshes([[verts4,verts5]],[[faces4,faces1]],[1,2])

##############################################################################

## Discusstion for 2.2

The left figure is the final mesh, and the right one is the target mesh. 

The tail of the final mesh is not as symmetric as that of the target mesh. ~~Additionally, the left part of the tail from the target mesh has obvious artifacts; these can be easily identified even without comparison.~~ Meanwhile, the belly of the final mesh has prominent lines on each side, in contrast to the smooth belly of the target mesh. Also, the dorsal fin on the final mesh is not sharp and appears to have lost a corner compared to the target mesh.

In conclusion, the optimized (final) mesh resembles a dolphin, though some details are lost.

In [ ]:
##############################################################################
# Code for 2.3 part1
def plot_losses_log(loss_dict, size=(13,5)):
  fig = plt.figure(figsize=size)
  ax = fig.gca()
  for loss_name, loss_values in loss_dict.items():
    ax.plot(loss_values, label=loss_name)
  ax.legend(fontsize="16")
  ax.set_xlabel("Iteration", fontsize="16")
  ax.set_ylabel("Loss", fontsize="16")
  ax.set_title("Loss vs iterations", fontsize="16")
  ax.set_yscale('log') 
losses_less = {"chamfer loss": chamfer_losses[250:],
          "edge loss": edge_losses[250:],
          "normal loss": normal_losses[250:],
          "laplacian loss": laplacian_losses[250:]}
plot_losses_log(losses, (18, 10))
##################################################################################

In [ ]:
# code for 2.3 part2
# The shape of the deform parameters is equal to the total number of vertices in src_mesh
deform_verts = torch.full(src_mesh.verts_packed().shape, 0.0, device=device, requires_grad=True)
# The optimizer
optimizer = torch.optim.SGD([deform_verts], lr=1.0, momentum=0.9)
# Number of optimization steps
Niter = 4000
# Weight for the chamfer loss
w_chamfer = 1.0
# Weight for mesh edge loss
w_edge = 1.0
# Weight for mesh normal consistency
w_normal = 0.01
# Weight for mesh laplacian smoothing
w_laplacian = 0.1
# Plot period for the losses
plot_period = 2000
loop = tqdm(range(Niter))

chamfer_losses = []
laplacian_losses = []
edge_losses = []
normal_losses = []
total_losses=[]
%matplotlib inline

for i in loop:
    # Initialize optimizer
    optimizer.zero_grad()

    # Deform the mesh
    new_src_mesh = src_mesh.offset_verts(deform_verts)

    # We sample 5k points from the surface of each mesh
    sample_trg = sample_points_from_meshes(trg_mesh, 5000)
    sample_src = sample_points_from_meshes(new_src_mesh, 5000)

    # We compare the two sets of pointclouds by computing (a) the chamfer loss
    loss_chamfer, _ = chamfer_distance(sample_trg, sample_src)

    # and (b) the edge length of the predicted mesh
    loss_edge = mesh_edge_loss(new_src_mesh)

    # mesh normal consistency
    loss_normal = mesh_normal_consistency(new_src_mesh)

    # mesh laplacian smoothing
    loss_laplacian = mesh_laplacian_smoothing(new_src_mesh, method="uniform")

    # Weighted sum of the losses
    loss = loss_chamfer * w_chamfer + loss_edge * w_edge + loss_normal * w_normal + loss_laplacian * w_laplacian

    # Print the losses
    loop.set_description('total_loss = %.6f' % loss)

    # Save the losses for plotting
    chamfer_losses.append(float(loss_chamfer.detach().cpu()))
    edge_losses.append(float(loss_edge.detach().cpu()))
    normal_losses.append(float(loss_normal.detach().cpu()))
    laplacian_losses.append(float(loss_laplacian.detach().cpu()))
    total_losses.append(float(loss.detach().cpu()))

    # Plot mesh
    if i % plot_period == 0:
        plot_pointcloud(new_src_mesh, title="iter: %d" % i)

    if i == 2000:
      for param_group in optimizer.param_groups:
        param_group['lr'] /= 2

    # Optimization step
    loss.backward()
    optimizer.step()

final_verts, final_faces = new_src_mesh.get_mesh_verts_faces(0)
final_verts_nback=final_verts*largest_distance
#final_obj = os.path.join('./', 'final_model3.obj')
#save_obj(final_obj, final_verts_nback, final_faces)
##############################################################################

In [ ]:
# code for 2.3 part3 (visualize the loss)
losses = {"chamfer loss": chamfer_losses,
          "edge loss": edge_losses,
          "normal loss": normal_losses,
          "laplacian loss": laplacian_losses,
          "total loss":total_losses}
plot_losses_log(losses, (18, 10))

In [ ]:
# code for 2.3 part 4 (visualize the mesh)
verts6=final_verts_nback.detach().cpu()
faces6=final_faces.cpu()
#fig5=make_subplots(1,3,specs=[[{"type": "mesh3d"}, {"type": "mesh3d"}],[{"type": "mesh3d"}]])
#fig5.add_trace(go.Mesh3d(x=verts4[:,0], y=verts4[:,1], z=verts4[:,2],i=faces4[:, 0],j=faces4[:, 1],k=faces4[:, 2]), row=1, col=1)
#fig5.add_trace(go.Mesh3d(x=verts6[:,0], y=verts6[:,1], z=verts6[:,2],i=faces6[:, 0],j=faces6[:, 1],k=faces6[:, 2]), row=1, col=2)
#fig5.add_trace(go.Mesh3d(x=verts5[:,0], y=verts5[:,1], z=verts5[:,2],i=faces1[:, 0],j=faces1[:, 1],k=faces1[:, 2]), row=1, col=3)
fig5=plotly_submeshes([[verts4,verts6,verts5]],[[faces4,faces6,faces1]],[1,3])
fig5.show()

## Discussion for 2.3

Above, to further observe the detaied loss variation, we plot the previous loss figure with the y coordinate scaling in logarithm. At the beginning of optimization process, the total loss and chamfer loss has a very high value and decline rapidly. After 1000 iterations, the total loss converges around 10^-2 scale, however, with oscillation.

Even though the final mesh (the blue mesh) has a very similar shape to the target mesh, I think the loss could even converge further. To be specific, after 2000 iterations, the final mesh shape is reasonable to human eye. On the other side, further optimization could achieve a more authentic and detailed shape.

Thus, I decided to run the optimization with more iterations and a decaying learning rate. The first figure in mesh visualization shows the results at 2000 iterations, and the second at 4000 iterations (the red mesh). Visually, these figures don't make a difference. However, the loss reached  a lower value (0.002287), which demonstrates our hypothesis that the loss could converge more.

In [ ]:
##############################################################################
# code for 2.4 part1 (optimization)
# The shape of the deform parameters is equal to the total number of vertices in src_mesh
deform_verts = torch.full(src_mesh.verts_packed().shape, 0.0, device=device, requires_grad=True)
# The optimizer
optimizer = torch.optim.SGD([deform_verts], lr=1.0, momentum=0.9)
# Number of optimization steps
Niter = 2000
# Weight for the chamfer loss
w_chamfer = 1.0
# Weight for mesh edge loss
w_edge = 0.0
# Weight for mesh normal consistency
w_normal =  0.0
# Weight for mesh laplacian smoothing
w_laplacian =  0.0
# Plot period for the losses
plot_period = 1000
loop = tqdm(range(Niter))

chamfer_losses = []
laplacian_losses = []
edge_losses = []
normal_losses = []
total_losses=[]
%matplotlib inline

for i in loop:
    # Initialize optimizer
    optimizer.zero_grad()

    # Deform the mesh
    new_src_mesh = src_mesh.offset_verts(deform_verts)

    # We sample 5k points from the surface of each mesh
    sample_trg = sample_points_from_meshes(trg_mesh, 5000)
    sample_src = sample_points_from_meshes(new_src_mesh, 5000)

    # We compare the two sets of pointclouds by computing (a) the chamfer loss
    loss_chamfer, _ = chamfer_distance(sample_trg, sample_src)

    # and (b) the edge length of the predicted mesh
    loss_edge = mesh_edge_loss(new_src_mesh)

    # mesh normal consistency
    loss_normal = mesh_normal_consistency(new_src_mesh)

    # mesh laplacian smoothing
    loss_laplacian = mesh_laplacian_smoothing(new_src_mesh, method="uniform")

    # Weighted sum of the losses
    loss = loss_chamfer * w_chamfer + loss_edge * w_edge + loss_normal * w_normal + loss_laplacian * w_laplacian

    # Print the losses
    loop.set_description('total_loss = %.6f' % loss)

    # Save the losses for plotting
    chamfer_losses.append(float(loss_chamfer.detach().cpu()))
    edge_losses.append(float(loss_edge.detach().cpu()))
    normal_losses.append(float(loss_normal.detach().cpu()))
    laplacian_losses.append(float(loss_laplacian.detach().cpu()))
    total_losses.append(float(loss.detach().cpu()))

    # Plot mesh
    if i % plot_period == 0:
        plot_pointcloud(new_src_mesh, title="iter: %d" % i)

    if i == 2000:
      for param_group in optimizer.param_groups:
        param_group['lr'] /= 2

    # Optimization step
    loss.backward()
    optimizer.step()

final_verts, final_faces = new_src_mesh.get_mesh_verts_faces(0)
final_verts_nback=final_verts*largest_distance
#final_obj = os.path.join('./', 'final_model3.obj')
#save_obj(final_obj, final_verts_nback, final_faces)
verts7=final_verts_nback.detach().cpu()
faces7=final_faces.cpu()

##############################################################################

In [ ]:
##############################################################################
# code for 2.4 part 2 (visualization)
plotly_submeshes([[verts7,verts5]],[[faces7,faces1]],(1,2))

## Discussion for 2.4

The loss converges much more rapidly than expected when optimization is constrained by only one type of loss. 

However, using only chamfer loss results in a completely unsuccessful mesh fit. The final mesh displays a crude and uneven dolphin shape, with the tips of the fins and the mouth becoming diffuse, pinprick-like shapes. It is difficult to identify the species from the mesh.

In [ ]:
##############################################################################
# code for E.1
def weight_experiment(src_mesh,trg_mesh,chamfer,edge,normal,laplacian,iterations):
    # The shape of the deform parameters is equal to the total number of vertices in src_mesh
    deform_verts = torch.full(src_mesh.verts_packed().shape, 0.0, device=device, requires_grad=True)
    # The optimizer
    optimizer = torch.optim.SGD([deform_verts], lr=1.0, momentum=0.9)
    # Number of optimization steps
    Niter = iterations
    # Weight for the chamfer loss
    w_chamfer = chamfer
    # Weight for mesh edge loss
    w_edge = edge
    # Weight for mesh normal consistency
    w_normal = normal
    # Weight for mesh laplacian smoothing
    w_laplacian = laplacian
    # Plot period for the losses
    plot_period = 1000
    loop = tqdm(range(Niter))

    chamfer_losses = []
    laplacian_losses = []
    edge_losses = []
    normal_losses = []
    total_losses = []
    %matplotlib inline

    for i in loop:
        # Initialize optimizer
        optimizer.zero_grad()

        # Deform the mesh
        new_src_mesh = src_mesh.offset_verts(deform_verts)

        # We sample 5k points from the surface of each mesh
        sample_trg = sample_points_from_meshes(trg_mesh, 5000)
        sample_src = sample_points_from_meshes(new_src_mesh, 5000)

        # We compare the two sets of pointclouds by computing (a) the chamfer loss
        loss_chamfer, _ = chamfer_distance(sample_trg, sample_src)

        # and (b) the edge length of the predicted mesh
        loss_edge = mesh_edge_loss(new_src_mesh)

        # mesh normal consistency
        loss_normal = mesh_normal_consistency(new_src_mesh)

        # mesh laplacian smoothing
        loss_laplacian = mesh_laplacian_smoothing(new_src_mesh, method="uniform")

        # Weighted sum of the losses
        loss = loss_chamfer * w_chamfer + loss_edge * w_edge + loss_normal * w_normal + loss_laplacian * w_laplacian

        # Print the losses
        loop.set_description('total_loss = %.6f' % loss)

        # Save the losses for plotting
        chamfer_losses.append(float(loss_chamfer.detach().cpu()))
        edge_losses.append(float(loss_edge.detach().cpu()))
        normal_losses.append(float(loss_normal.detach().cpu()))
        laplacian_losses.append(float(loss_laplacian.detach().cpu()))
        total_losses.append(float(loss.detach().cpu()))

        # Plot mesh
        if i % plot_period == 0:
            plot_pointcloud(new_src_mesh, title="iter: %d" % i)

        if i == 2000:
            for param_group in optimizer.param_groups:
                param_group['lr'] /= 2

        # Optimization step
        loss.backward()
        optimizer.step()

    final_verts, final_faces = new_src_mesh.get_mesh_verts_faces(0)
    return final_verts, final_faces
##############################################################################

In [ ]:
# code for E.1 part 1 (edge length weight)
final_verts,final_faces = weight_experiment(src_mesh,trg_mesh,1.0,0.5,0.01,0.1,2000)

final_verts_nback = final_verts * largest_distance
#final_obj = os.path.join('./', 'final_model3.obj')
#save_obj(final_obj, final_verts_nback, final_faces)
verts8 = final_verts_nback.detach().cpu()
faces8 = final_faces.cpu()

final_verts,final_faces =weight_experiment(src_mesh,trg_mesh,1.0,2.0,0.01,0.1,2000)
final_verts_nback = final_verts * largest_distance
#final_obj = os.path.join('./', 'final_model3.obj')
#save_obj(final_obj, final_verts_nback, final_faces)
verts9 = final_verts_nback.detach().cpu()
faces9 = final_faces.cpu()

In [ ]:
##############################################################################
# code for E.1 part 1 (visualization)
plotly_submeshes([[verts8,verts9,verts5]],[[faces8,faces9,faces1]],(1,3))

## Discussion for E.1 part 1 (experiment on edge length weight)

Above, the first figure (the blue mesh) represents the mesh optimized with an edge length weight of 0.5, the second with a weight of 2.0, and the third is the target mesh.

We observed that the first mesh is not smooth enough to be identified as a dolphin, suffering from stretching artifacts in mouth part and at the tip of tail. This indicates that the low edge length loss allows longer egdes exists, and further forms those streching artifacts.

For the second mesh (the red mesh), it almost achieves to be a good shape of a dolphin. Nevertheless, the left part of the dolphin has a little artifacts, failing to maintain the overall smooth of the optimizied mesh. We can infer that the spike shape artifacts is caused by the high edge length loss which enforcing shorter edges.

In [ ]:
# code for E.1 part 2 (normal consistency weight)
final_verts,final_faces = weight_experiment(src_mesh,trg_mesh,1.0,1.0,0.02,0.1,2000)
 
final_verts_nback = final_verts * largest_distance
#final_obj = os.path.join('./', 'final_model3.obj')
#save_obj(final_obj, final_verts_nback, final_faces)
verts10 = final_verts_nback.detach().cpu()
faces10 = final_faces.cpu()

final_verts,final_faces =weight_experiment(src_mesh,trg_mesh,1.0,1.0,0.005,0.1,2000)
final_verts_nback = final_verts * largest_distance
#final_obj = os.path.join('./', 'final_model3.obj')
#save_obj(final_obj, final_verts_nback, final_faces)
verts11 = final_verts_nback.detach().cpu()
faces11 = final_faces.cpu()

In [ ]:
##############################################################################
# code for E.1 part 2 (visualization)
plotly_submeshes([[verts10,verts11,verts5]],[[faces10,faces11,faces1]],(1,3))

## Discussion for E.1 part 2 (experiment on normal consistency weight)

~~We know that normal consistency is a local smooth constraint, and we expect that larger weight of normal consistency makes the mesh more smooth while lower has opposite effect. However the results of experienment are not as expected. Both optimized meshes characterize obvious artifacts on the tail part. We think that the first optimized mesh with larger weight of normal consistency needs more iteration to make sure conversion. Thus we decided to optimize with the same configuration again except through 4000 iterations. The optimization is shown below.~~

Note: The discussion above is based on another optimization process. However, with some randomness, 2000 iterations seems to be enough for normal conssitency weight loss experiment, you can check the discussion in following discussion section. 

In [ ]:
# code for E.1 part 2 (normal consistency weight, 4000 iterations)
final_verts,final_faces = weight_experiment(src_mesh,trg_mesh,1.0,1.0,0.02,0.1,4000)
 
final_verts_nback = final_verts * largest_distance
#final_obj = os.path.join('./', 'final_model3.obj')
#save_obj(final_obj, final_verts_nback, final_faces)
verts12 = final_verts_nback.detach().cpu()
faces12 = final_faces.cpu()

final_verts,final_faces =weight_experiment(src_mesh,trg_mesh,1.0,1.0,0.005,0.1,4000)
final_verts_nback = final_verts * largest_distance
#final_obj = os.path.join('./', 'final_model3.obj')
#save_obj(final_obj, final_verts_nback, final_faces)
verts13 = final_verts_nback.detach().cpu()
faces13 = final_faces.cpu()

In [ ]:
##############################################################################
# code for E.1 part 2 (visualization)
plotly_submeshes([[verts12,verts13,verts5]],[[faces12,faces13,faces1]],(1,3))

## Discussion for E.1 part 2 continued (experiment on normal consistency weight)

The final meshes are now in the perfect shape of a dolphin. The only difference we find is that the dorsal fin of the first mesh is smoother. This proves that the lower the normal consistency loss, the smoother the optimized mesh is.

In [ ]:
# code for E.1 part 3 (laplacian smoothing weight)
final_verts,final_faces = weight_experiment(src_mesh,trg_mesh,1.0,1.0,0.01,0.2,2000)
 
final_verts_nback = final_verts * largest_distance
#final_obj = os.path.join('./', 'final_model3.obj')
#save_obj(final_obj, final_verts_nback, final_faces)
verts14 = final_verts_nback.detach().cpu()
faces14 = final_faces.cpu()

final_verts,final_faces =weight_experiment(src_mesh,trg_mesh,1.0,1.0,0.01,0.05,2000)
final_verts_nback = final_verts * largest_distance
#final_obj = os.path.join('./', 'final_model3.obj')
#save_obj(final_obj, final_verts_nback, final_faces)
verts15 = final_verts_nback.detach().cpu()
faces15 = final_faces.cpu()

In [ ]:
##############################################################################
# code for E.1 part 3 (visualization)
plotly_submeshes([[verts14,verts15,verts5]],[[faces14,faces15,faces1]],(1,3))

## Discussion for E.1 part 3 (experiment on laplacian smoothing) 

The first figure is optimized by doubled laplacian smoothing weight while the second by halved weight.

The laplacian smoothing is a local smooth factor similar to normal consistency. The first figure is oversmoothed while the second figure seems to have a relatively rough surface.

In [ ]:
# code for E.1 part 4 (chamfer loss weight)
final_verts,final_faces = weight_experiment(src_mesh,trg_mesh,2.0,1.0,0.01,0.1,2000)
 
final_verts_nback = final_verts * largest_distance
#final_obj = os.path.join('./', 'final_model3.obj')
#save_obj(final_obj, final_verts_nback, final_faces)
verts16 = final_verts_nback.detach().cpu()
faces16 = final_faces.cpu()

final_verts,final_faces =weight_experiment(src_mesh,trg_mesh,0.5,1.0,0.01,0.1,2000)
final_verts_nback = final_verts * largest_distance
#final_obj = os.path.join('./', 'final_model3.obj')
#save_obj(final_obj, final_verts_nback, final_faces)
verts17 = final_verts_nback.detach().cpu()
faces17 = final_faces.cpu()

In [ ]:
##############################################################################
# code for E.1 part 4 (visualization)
plotly_submeshes([[verts16,verts17,verts5]],[[faces16,faces17,faces1]],(1,3))

## Discussion for E.1 part 4 (experiment on chamfer loss)

The chamfer loss is a reconstruction loss that is essential in constructing a dolphin shape. The first figure is generated with doubled chamfer loss weight, and the second with halved chamfer loss weight.

With lower chamfer loss, the first figure maintains a better dolphin shape.

## Conclusion for E.1

Well-designed loss weights are constraints that play an important role in constructing a good and accurate mesh. We have experimented with different configurations by changing a single weight to optimize a new mesh. Changing a single weight without considering other weights creates imbalance among those constraints, and that is the main reason most of these experiments show dolphin meshes with flaws.

In [ ]:
##############################################################################
# code for 2.5
def SGD_opt(src_mesh,trg_mesh,chamfer,edge,normal,laplacian,iterations,lr,momentum):
    # The shape of the deform parameters is equal to the total number of vertices in src_mesh
    deform_verts = torch.full(src_mesh.verts_packed().shape, 0.0, device=device, requires_grad=True)
    # The optimizer
    optimizer = torch.optim.SGD([deform_verts], lr=lr, momentum=momentum)
    # Number of optimization steps
    Niter = iterations
    # Weight for the chamfer loss
    w_chamfer = chamfer
    # Weight for mesh edge loss
    w_edge = edge
    # Weight for mesh normal consistency
    w_normal = normal
    # Weight for mesh laplacian smoothing
    w_laplacian = laplacian
    # Plot period for the losses
    plot_period = 1000
    loop = tqdm(range(Niter))

    chamfer_losses = []
    laplacian_losses = []
    edge_losses = []
    normal_losses = []
    total_losses = []
    %matplotlib inline

    for i in loop:
        # Initialize optimizer
        optimizer.zero_grad()

        # Deform the mesh
        new_src_mesh = src_mesh.offset_verts(deform_verts)

        # We sample 5k points from the surface of each mesh
        sample_trg = sample_points_from_meshes(trg_mesh, 5000)
        sample_src = sample_points_from_meshes(new_src_mesh, 5000)

        # We compare the two sets of pointclouds by computing (a) the chamfer loss
        loss_chamfer, _ = chamfer_distance(sample_trg, sample_src)

        # and (b) the edge length of the predicted mesh
        loss_edge = mesh_edge_loss(new_src_mesh)

        # mesh normal consistency
        loss_normal = mesh_normal_consistency(new_src_mesh)

        # mesh laplacian smoothing
        loss_laplacian = mesh_laplacian_smoothing(new_src_mesh, method="uniform")

        # Weighted sum of the losses
        loss = loss_chamfer * w_chamfer + loss_edge * w_edge + loss_normal * w_normal + loss_laplacian * w_laplacian

        # Print the losses
        loop.set_description('total_loss = %.6f' % loss)

        # Save the losses for plotting
        chamfer_losses.append(float(loss_chamfer.detach().cpu()))
        edge_losses.append(float(loss_edge.detach().cpu()))
        normal_losses.append(float(loss_normal.detach().cpu()))
        laplacian_losses.append(float(loss_laplacian.detach().cpu()))
        total_losses.append(float(loss.detach().cpu()))

        # Plot mesh
        if i % plot_period == 0:
            plot_pointcloud(new_src_mesh, title="iter: %d" % i)

        if i == 2000:
            for param_group in optimizer.param_groups:
                param_group['lr'] /= 2

        # Optimization step
        loss.backward()
        optimizer.step()

    final_verts, final_faces = new_src_mesh.get_mesh_verts_faces(0)
    losses = {"chamfer loss": chamfer_losses,
          "edge loss": edge_losses,
          "normal loss": normal_losses,
          "laplacian loss": laplacian_losses,
          "total loss":total_losses}

    return final_verts, final_faces,losses
    
def Adam_opt(src_mesh,trg_mesh,chamfer,edge,normal,laplacian,iterations,lr,betas):
    # The shape of the deform parameters is equal to the total number of vertices in src_mesh
    deform_verts = torch.full(src_mesh.verts_packed().shape, 0.0, device=device, requires_grad=True)
    # The optimizer
    optimizer = torch.optim.Adam([deform_verts], lr=lr, betas=betas)
    # Number of optimization steps
    Niter = iterations
    # Weight for the chamfer loss
    w_chamfer = chamfer
    # Weight for mesh edge loss
    w_edge = edge
    # Weight for mesh normal consistency
    w_normal = normal
    # Weight for mesh laplacian smoothing
    w_laplacian = laplacian
    # Plot period for the losses
    plot_period = 1000
    loop = tqdm(range(Niter))

    chamfer_losses = []
    laplacian_losses = []
    edge_losses = []
    normal_losses = []
    total_losses = []
    %matplotlib inline

    for i in loop:
        # Initialize optimizer
        optimizer.zero_grad()

        # Deform the mesh
        new_src_mesh = src_mesh.offset_verts(deform_verts)

        # We sample 5k points from the surface of each mesh
        sample_trg = sample_points_from_meshes(trg_mesh, 5000)
        sample_src = sample_points_from_meshes(new_src_mesh, 5000)

        # We compare the two sets of pointclouds by computing (a) the chamfer loss
        loss_chamfer, _ = chamfer_distance(sample_trg, sample_src)

        # and (b) the edge length of the predicted mesh
        loss_edge = mesh_edge_loss(new_src_mesh)

        # mesh normal consistency
        loss_normal = mesh_normal_consistency(new_src_mesh)

        # mesh laplacian smoothing
        loss_laplacian = mesh_laplacian_smoothing(new_src_mesh, method="uniform")

        # Weighted sum of the losses
        loss = loss_chamfer * w_chamfer + loss_edge * w_edge + loss_normal * w_normal + loss_laplacian * w_laplacian

        # Print the losses
        loop.set_description('total_loss = %.6f' % loss)

        # Save the losses for plotting
        chamfer_losses.append(float(loss_chamfer.detach().cpu()))
        edge_losses.append(float(loss_edge.detach().cpu()))
        normal_losses.append(float(loss_normal.detach().cpu()))
        laplacian_losses.append(float(loss_laplacian.detach().cpu()))
        total_losses.append(float(loss.detach().cpu()))

        # Plot mesh
        if i % plot_period == 0:
            plot_pointcloud(new_src_mesh, title="iter: %d" % i)

        if i == 2000:
            for param_group in optimizer.param_groups:
                param_group['lr'] /= 2

        # Optimization step
        loss.backward()
        optimizer.step()

    final_verts, final_faces = new_src_mesh.get_mesh_verts_faces(0)
    losses = {"chamfer loss": chamfer_losses,
          "edge loss": edge_losses,
          "normal loss": normal_losses,
          "laplacian loss": laplacian_losses,
          "total loss":total_losses}

    return final_verts, final_faces,losses

def RMSprop_opt(src_mesh,trg_mesh,chamfer,edge,normal,laplacian,iterations,lr,momentum):
    # The shape of the deform parameters is equal to the total number of vertices in src_mesh
    deform_verts = torch.full(src_mesh.verts_packed().shape, 0.0, device=device, requires_grad=True)
    # The optimizer
    optimizer = torch.optim.RMSprop([deform_verts], lr=lr, momentum=momentum)
    # Number of optimization steps
    Niter = iterations
    # Weight for the chamfer loss
    w_chamfer = chamfer
    # Weight for mesh edge loss
    w_edge = edge
    # Weight for mesh normal consistency
    w_normal = normal
    # Weight for mesh laplacian smoothing
    w_laplacian = laplacian
    # Plot period for the losses
    plot_period = 1000
    loop = tqdm(range(Niter))

    chamfer_losses = []
    laplacian_losses = []
    edge_losses = []
    normal_losses = []
    total_losses = []
    %matplotlib inline

    for i in loop:
        # Initialize optimizer
        optimizer.zero_grad()

        # Deform the mesh
        new_src_mesh = src_mesh.offset_verts(deform_verts)

        # We sample 5k points from the surface of each mesh
        sample_trg = sample_points_from_meshes(trg_mesh, 5000)
        sample_src = sample_points_from_meshes(new_src_mesh, 5000)

        # We compare the two sets of pointclouds by computing (a) the chamfer loss
        loss_chamfer, _ = chamfer_distance(sample_trg, sample_src)

        # and (b) the edge length of the predicted mesh
        loss_edge = mesh_edge_loss(new_src_mesh)

        # mesh normal consistency
        loss_normal = mesh_normal_consistency(new_src_mesh)

        # mesh laplacian smoothing
        loss_laplacian = mesh_laplacian_smoothing(new_src_mesh, method="uniform")

        # Weighted sum of the losses
        loss = loss_chamfer * w_chamfer + loss_edge * w_edge + loss_normal * w_normal + loss_laplacian * w_laplacian

        # Print the losses
        loop.set_description('total_loss = %.6f' % loss)

        # Save the losses for plotting
        chamfer_losses.append(float(loss_chamfer.detach().cpu()))
        edge_losses.append(float(loss_edge.detach().cpu()))
        normal_losses.append(float(loss_normal.detach().cpu()))
        laplacian_losses.append(float(loss_laplacian.detach().cpu()))
        total_losses.append(float(loss.detach().cpu()))

        # Plot mesh
        if i % plot_period == 0:
            plot_pointcloud(new_src_mesh, title="iter: %d" % i)

        if i == 2000:
            for param_group in optimizer.param_groups:
                param_group['lr'] /= 2

        # Optimization step
        loss.backward()
        optimizer.step()
    losses = {"chamfer loss": chamfer_losses,
          "edge loss": edge_losses,
          "normal loss": normal_losses,
          "laplacian loss": laplacian_losses,
          "total loss":total_losses}
    final_verts, final_faces = new_src_mesh.get_mesh_verts_faces(0)
    return final_verts, final_faces,losses
##############################################################################

In [ ]:
# code for 2.5 part 1 (normal config)
final_verts, final_faces,losses1=SGD_opt(src_mesh,trg_mesh,1.0,1.0,0.01,0.1,2000,1.0,0.9)
final_verts_nback = final_verts * largest_distance
#final_obj = os.path.join('./', 'final_model3.obj')
#save_obj(final_obj, final_verts_nback, final_faces)
verts18 = final_verts_nback.detach().cpu()
faces18 = final_faces.cpu()

final_verts, final_faces,losses2=Adam_opt(src_mesh,trg_mesh,1.0,1.0,0.01,0.1,2000,1.0,(0.9,0.999))
final_verts_nback = final_verts * largest_distance
#final_obj = os.path.join('./', 'final_model3.obj')
#save_obj(final_obj, final_verts_nback, final_faces)
verts19 = final_verts_nback.detach().cpu()
faces19 = final_faces.cpu()

final_verts, final_faces,losses3=RMSprop_opt(src_mesh,trg_mesh,1.0,1.0,0.01,0.1,2000,1.0,0.9)
final_verts_nback = final_verts * largest_distance
#final_obj = os.path.join('./', 'final_model3.obj')
#save_obj(final_obj, final_verts_nback, final_faces)
verts20 = final_verts_nback.detach().cpu()
faces20 = final_faces.cpu()

In [ ]:
plot_losses(losses1,(18,10))
plot_losses(losses2,(18,10))
plot_losses(losses3,(18,10))

In [ ]:
plotly_submeshes([[verts18,verts19,verts20]],[[faces18,faces19,faces20]],(1,3))

## Discussion for 2.5 part 1 (normal config)

The first loss figure and the first mesh represent the result from SGD, the second pair represent the result from Adam, and the third pair represent the result from RMSprop. 

The learning rate of 1.0 seems to be effective only for SGD method, while the Adam and the RMSprop method cannot reconstruct mesh in such configuration.

In [ ]:
# code for 2.5 part 2 (learning rate)
final_verts, final_faces,losses4=SGD_opt(src_mesh,trg_mesh,1.0,1.0,0.01,0.1,2000,0.1,0.9)
final_verts_nback = final_verts * largest_distance
#final_obj = os.path.join('./', 'final_model3.obj')
#save_obj(final_obj, final_verts_nback, final_faces)
verts21 = final_verts_nback.detach().cpu()
faces21 = final_faces.cpu()

final_verts, final_faces,losses5=Adam_opt(src_mesh,trg_mesh,1.0,1.0,0.01,0.1,2000,0.001,(0.9,0.999))
final_verts_nback = final_verts * largest_distance
#final_obj = os.path.join('./', 'final_model3.obj')
#save_obj(final_obj, final_verts_nback, final_faces)
verts22 = final_verts_nback.detach().cpu()
faces22 = final_faces.cpu()

final_verts, final_faces,losses6=RMSprop_opt(src_mesh,trg_mesh,1.0,1.0,0.01,0.1,2000,0.0001,0.9)
final_verts_nback = final_verts * largest_distance
#final_obj = os.path.join('./', 'final_model3.obj')
#save_obj(final_obj, final_verts_nback, final_faces)
verts23 = final_verts_nback.detach().cpu()
faces23 = final_faces.cpu()

In [ ]:
plot_losses_log(losses4,(18,10))
plot_losses_log(losses5,(18,10))
plot_losses_log(losses6,(18,10))

In [ ]:
plotly_submeshes([[verts21,verts22,verts23]],[[faces21,faces22,faces23]],(1,3))

## Discussion for 2.5 part 2 (learning rate)

Above, to make sure the output mesh is not non-sense, we have tried different learning rates to for these 3 methods. We have shown 1 setting per method. 

The first pair, SGD, has the largest learning rate of 0.1. However, this learning rate is not enough for the mesh to converge to a dolphin shape. So, the SGD converges slowest. 

The second pair, Adam, has the learning rate of 0.001. The final mesh from this setting presents some detail of a dolphin. By observing the loss curve, the loss does not converge to its best situtation. And more iterations or better learning rate would improve the result. Compared to the first and the third method, the Adam converges in a moderate speed.

The third pair, RMSprop, has the smallest learning rate of 0.0001. It has the best reconstruction shape among these 3 methods. And its loss has converged below 10-3 scale which makes it lowest loss among these 3 experiment. We can conclude that RMSprop method has the fastest converging speed and the best loss, in 2000 iterations.

In [ ]:
# code for 2.5 part 3 (momentum)
final_verts, final_faces,losses7=SGD_opt(src_mesh,trg_mesh,1.0,1.0,0.01,0.1,2000,0.1,0.5)
final_verts_nback = final_verts * largest_distance
#final_obj = os.path.join('./', 'final_model3.obj')
#save_obj(final_obj, final_verts_nback, final_faces)
verts24 = final_verts_nback.detach().cpu()
faces24 = final_faces.cpu()

final_verts, final_faces,losses8=Adam_opt(src_mesh,trg_mesh,1.0,1.0,0.01,0.1,2000,0.001,(0.5,0.999))
final_verts_nback = final_verts * largest_distance
#final_obj = os.path.join('./', 'final_model3.obj')
#save_obj(final_obj, final_verts_nback, final_faces)
verts25 = final_verts_nback.detach().cpu()
faces25 = final_faces.cpu()

final_verts, final_faces,losses9=RMSprop_opt(src_mesh,trg_mesh,1.0,1.0,0.01,0.1,2000,0.0005,0.5)
final_verts_nback = final_verts * largest_distance
#final_obj = os.path.join('./', 'final_model3.obj')
#save_obj(final_obj, final_verts_nback, final_faces)
verts26 = final_verts_nback.detach().cpu()
faces26 = final_faces.cpu()

In [ ]:
plot_losses_log(losses7,(18,10))
plot_losses_log(losses8,(18,10))
plot_losses_log(losses9,(18,10))

In [ ]:
plotly_submeshes([[verts24,verts25,verts26]],[[faces24,faces25,faces26]],(1,3))

## Discussion for 2.5 part 3 (momentum)

Above, we experiment these 3 methods with lower mementum.

# 3. Experimenting with Other Shapes

In [ ]:
# download the mug model - Mug by Microsoft is licensed under Creative Commons Attribution
# originally found at https://sketchfab.com/3d-models/mug-17c4808537f1448590378b3643c6da72
!wget -nc https://raw.githubusercontent.com/hallpaz/3dsystems21/main/data/mug.obj
!wget -nc https://github.com/alecjacobson/common-3d-test-models/blob/master/data/stanford-bunny.obj
#!wget -nc https://github.com/alecjacobson/common-3d-test-models/blob/master/data/beetle.obj
!wget -nc https://raw.githubusercontent.com/king0ffire/data/main/ring.obj

3.1 Is it possible to deform any mesh into another? Explain your thoughts.

3.2 Run the experiment again and try to deform a sphere into a mug.

3.3 Change the `src_mesh` to a `torus`. You can import the torus primitive from `pytorch3d.utils`. Now, try to deform the torus into a mug.


**[Extra] E.2 Run the experiments again using other shapes (either find and download shapes from the internet or create your own models).**

## Answer for 3.1

It is not possible to deform any mesh into another. However, we can add constraints to make this statement true. Topologically speaking, if two meshes share the same topology, they can be deformed into each other. That is to say, only homeomorphic meshes can be deformed into each other.

In [ ]:
##############################################################################
# Code for 3.2
verts,faces,_=load_obj("mug.obj")
faces_idx = faces.verts_idx.to(device)
verts = verts.to(device)
centered_verts = verts-verts.mean(dim=0)#
largest_distance=centered_verts.norm(dim=1).max()
normalized_verts = centered_verts/largest_distance
mug_mesh=Meshes([normalized_verts], [faces_idx])
verts31=normalized_verts.cpu()
faces31=faces_idx.cpu()

fig6 = go.Figure(data=[go.Mesh3d(x=verts2[:,0], y=verts2[:,1], z=verts2[:,2],i=faces2[:, 0],j=faces2[:, 1],k=faces2[:, 2],opacity=0.5),go.Mesh3d(x=verts31[:,0], y=verts31[:,1], z=verts31[:,2],i=faces31[:, 0],j=faces31[:, 1],k=faces31[:, 2],opacity=0.5)])
fig6.show()

final_verts, final_faces,losses10=RMSprop_opt(src_mesh,mug_mesh,1.0,1.0,0.01,0.1,2000,0.0001,0.9)
final_verts_nback = final_verts 
#final_obj = os.path.join('./', 'final_model3.obj')
#save_obj(final_obj, final_verts_nback, final_faces)
verts27 = final_verts_nback.detach().cpu()
faces27 = final_faces.cpu()
verts28 = mug_mesh.verts_packed().cpu()
faces28=mug_mesh.faces_packed().cpu()

##############################################################################

In [ ]:
plotly_submeshes([[verts27,verts28]],[[faces27,faces28]],(1,2))

In [ ]:
# Code for 3.3
verts,faces,_=load_obj("mug.obj")
faces_idx = faces.verts_idx.to(device)
verts = verts.to(device)
centered_verts = verts-verts.mean(dim=0)#
largest_distance=centered_verts.norm(dim=1).max()
normalized_verts = centered_verts/largest_distance+torch.tensor([-0.4,0.0,0.0],device=device)
mug_mesh=Meshes([normalized_verts], [faces_idx])
verts31=normalized_verts.cpu()
faces31=faces_idx.cpu()

torus_mesh=torus(0.65,0.7,100,100,device)

verts30=torus_mesh.verts_packed().cpu()
faces30=torus_mesh.faces_packed().cpu()

fig7 = go.Figure(data=[go.Mesh3d(x=verts30[:,0], y=verts30[:,1], z=verts30[:,2],i=faces30[:, 0],j=faces30[:, 1],k=faces30[:, 2],opacity=0.5),go.Mesh3d(x=verts31[:,0], y=verts31[:,1], z=verts31[:,2],i=faces31[:, 0],j=faces31[:, 1],k=faces31[:, 2],opacity=0.5)])
fig7.show()

final_verts, final_faces,losses11=RMSprop_opt(torus_mesh,mug_mesh,1.0,0.5,0.01,0.1,2000,0.0001,0.9)
final_verts_nback = final_verts 
#final_obj = os.path.join('./', 'final_model3.obj')
#save_obj(final_obj, final_verts_nback, final_faces)
verts29 = final_verts_nback.detach().cpu()
faces29 = final_faces.cpu()



In [ ]:
plotly_submeshes([[verts29,verts28]],[[faces29,faces28]],(1,2))

In [ ]:
# Code for E.2
verts,faces,_=load_obj("ring.obj")
faces_idx = faces.verts_idx.to(device)
verts = verts.to(device)
centered_verts = verts-verts.mean(dim=0)
rotate_transform = RotateAxisAngle(angle=90, axis="X").to(device)
centered_verts = rotate_transform.transform_points(centered_verts)
largest_distance=centered_verts.norm(dim=1).max()
normalized_verts = centered_verts/largest_distance+torch.tensor([0.0,-0.3,0.0],device=device)
normalized_verts = normalized_verts*1.5

ring_mesh=Meshes([normalized_verts], [faces_idx])
verts32=normalized_verts.cpu()  # 32 is normalized ring
faces32=faces_idx.cpu()

torus_mesh=torus(0.6,1.0,100,100,device)

verts30=torus_mesh.verts_packed().cpu()
faces30=torus_mesh.faces_packed().cpu()

fig8 = go.Figure(data=[go.Mesh3d(x=verts30[:,0], y=verts30[:,1], z=verts30[:,2],i=faces30[:, 0],j=faces30[:, 1],k=faces30[:, 2],opacity=0.5),go.Mesh3d(x=verts32[:,0], y=verts32[:,1], z=verts32[:,2],i=faces32[:, 0],j=faces32[:, 1],k=faces32[:, 2],opacity=0.5)])
fig8.show()

final_verts, final_faces,losses12=RMSprop_opt(torus_mesh,ring_mesh,1.0,1.0,0.01,0.1,2000,0.0001,0.9)
final_verts_nback = final_verts 
#final_obj = os.path.join('./', 'final_model3.obj')
#save_obj(final_obj, final_verts_nback, final_faces)
verts33 = final_verts_nback.detach().cpu()
faces33 = final_faces.cpu()



In [ ]:
plotly_submeshes([[verts33,verts32]],[[faces33,faces32]],(1,2))

## Discussion for 3.1-E.2

The results in Section 3.2-E.2 demonstrate our thoughts from Section 3.1. 

In the 'Code for 3.2' section, we tried to deform a sphere into a mug. The optimized mesh resembles a basic mug shape. However, the handle is not formed since no hole is observed inside the handle. 

In the 'Code for 3.3' section, we tried to deform a torus into a mug. Although the torus and mug share the same topology, with one hole, the optimized mesh has formed a handle with a hole inside it. However, since the mug is double-walled, it is very hard for the torus to stretch to fit the cup, resulting in a very uneven appearance. We have tried different configurations including the torus sides, learning rate, and loss weights, but it remains a very challenging task to perfectly deform the torus into a mug. Thus, we decided to deform the torus into other simpler topologically similar meshes with one hole. 

In the 'Code for E.2' section, we deformed the torus into a ring with a sphere-shaped gemstone on it. The resulting mesh (the blue mesh) has a very similar shape to the original ring (the red mesh). This successful deformation shows that homeomorphic meshes can be deformed into each other.

# 4. Food for Thoughts

Let's say you have a big set Ω of arbitrary meshes, and 1 mesh or a small set Δ of meshes, for example from the same class, that you are instered in. How would you solve the problem of fitting meshes from Ω to meshes in Δ?